In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    BBB_Train_Decile0 = pd.read_pickle("BBB_Train_Decile0.pkl")
    BBB_Train_Targets_1D = pd.read_pickle("BBB_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("BBB_Test_Decile0.pkl")
    y_test = pd.read_pickle("BBB_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(BBB_Train_Decile0, BBB_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150,200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM BBB Decile 0')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))
    

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3087/3087 [==============================] - 2s 622us/step - loss: 0.6216 - acc: 0.6699 - val_loss: 0.6008 - val_acc: 0.6865
Epoch 38/100
3087/3087 [==============================] - 2s 555us/step - loss: 0.6192 - acc: 0.6718 - val_loss: 0.6007 - val_acc: 0.6865
Epoch 39/100
3087/3087 [==============================] - 2s 576us/step - loss: 0.6204 - acc: 0.6709 - val_loss: 0.6001 - val_acc: 0.6865
Epoch 40/100
3087/3087 [==============================] - 2s 502us/step - loss: 0.6221 - acc: 0.6709 - val_loss: 0.6005 - val_acc: 0.6865
Epoch 41/100
3087/3087 [==============================] - 2s 641us/step - loss: 0.6225 - acc: 0.6712 - val_loss: 0.6005 - val_acc: 0.6865
Epoch 42/100
3087/3087 [==============================] - 2s 540us/step - loss: 0.6223 - acc: 0.6712 - val_loss: 0.6001 - val_acc: 0.6865

Epoch 00042: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 43/100
3087/3087 [==============================] - 2s 512us/step - loss: 0.6211 - acc: 0.6706 - val_loss: 0.6000 

Epoch 96/100
3087/3087 [==============================] - 2s 526us/step - loss: 0.6198 - acc: 0.6709 - val_loss: 0.5986 - val_acc: 0.6865
Epoch 97/100
3087/3087 [==============================] - 2s 517us/step - loss: 0.6197 - acc: 0.6709 - val_loss: 0.5985 - val_acc: 0.6865
Epoch 98/100
3087/3087 [==============================] - 1s 453us/step - loss: 0.6179 - acc: 0.6712 - val_loss: 0.5985 - val_acc: 0.6865
Epoch 99/100
3087/3087 [==============================] - 1s 444us/step - loss: 0.6207 - acc: 0.6709 - val_loss: 0.5984 - val_acc: 0.6865
Epoch 100/100
3087/3087 [==============================] - 1s 451us/step - loss: 0.6185 - acc: 0.6709 - val_loss: 0.5985 - val_acc: 0.6865
Test accuracy: 0.6865284974093264
Train on 3087 samples, validate on 772 samples
Epoch 1/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.6568 - acc: 0.6527 - val_loss: 0.6172 - val_acc: 0.6878
Epoch 2/150
3087/3087 [==============================] - 1s 434us/step - loss: 0.6313 - acc: 

3087/3087 [==============================] - 1s 442us/step - loss: 0.6136 - acc: 0.6709 - val_loss: 0.5834 - val_acc: 0.6956
Epoch 54/150
3087/3087 [==============================] - 1s 384us/step - loss: 0.6131 - acc: 0.6722 - val_loss: 0.5834 - val_acc: 0.6956
Epoch 55/150
3087/3087 [==============================] - 1s 393us/step - loss: 0.6142 - acc: 0.6731 - val_loss: 0.5834 - val_acc: 0.6956
Epoch 56/150
3087/3087 [==============================] - 1s 377us/step - loss: 0.6121 - acc: 0.6709 - val_loss: 0.5833 - val_acc: 0.6956
Epoch 57/150
3087/3087 [==============================] - 1s 414us/step - loss: 0.6121 - acc: 0.6725 - val_loss: 0.5833 - val_acc: 0.6956
Epoch 58/150
3087/3087 [==============================] - 1s 426us/step - loss: 0.6141 - acc: 0.6738 - val_loss: 0.5834 - val_acc: 0.6956
Epoch 59/150
3087/3087 [==============================] - 1s 422us/step - loss: 0.6148 - acc: 0.6725 - val_loss: 0.5833 - val_acc: 0.6956
Epoch 60/150
3087/3087 [=======================

3087/3087 [==============================] - 1s 255us/step - loss: 0.6098 - acc: 0.6722 - val_loss: 0.5830 - val_acc: 0.6956
Epoch 113/150
3087/3087 [==============================] - 1s 408us/step - loss: 0.6127 - acc: 0.6725 - val_loss: 0.5830 - val_acc: 0.6956
Epoch 114/150
3087/3087 [==============================] - 1s 377us/step - loss: 0.6145 - acc: 0.6728 - val_loss: 0.5830 - val_acc: 0.6956
Epoch 115/150
3087/3087 [==============================] - 1s 370us/step - loss: 0.6122 - acc: 0.6722 - val_loss: 0.5830 - val_acc: 0.6956
Epoch 116/150
3087/3087 [==============================] - 1s 410us/step - loss: 0.6147 - acc: 0.6715 - val_loss: 0.5830 - val_acc: 0.6956
Epoch 117/150
3087/3087 [==============================] - 1s 414us/step - loss: 0.6133 - acc: 0.6722 - val_loss: 0.5830 - val_acc: 0.6956
Epoch 118/150
3087/3087 [==============================] - 1s 396us/step - loss: 0.6117 - acc: 0.6728 - val_loss: 0.5830 - val_acc: 0.6956
Epoch 119/150
3087/3087 [================

...

Epoch 7/200
3087/3087 [==============================] - 1s 480us/step - loss: 0.6198 - acc: 0.6712 - val_loss: 0.5935 - val_acc: 0.6865
Epoch 8/200
3087/3087 [==============================] - 1s 477us/step - loss: 0.6176 - acc: 0.6702 - val_loss: 0.5919 - val_acc: 0.6865
Epoch 9/200
3087/3087 [==============================] - 2s 497us/step - loss: 0.6212 - acc: 0.6699 - val_loss: 0.5938 - val_acc: 0.6878
Epoch 10/200
3087/3087 [==============================] - 1s 471us/step - loss: 0.6167 - acc: 0.6709 - val_loss: 0.5953 - val_acc: 0.6904
Epoch 11/200
3087/3087 [==============================] - 1s 463us/step - loss: 0.6186 - acc: 0.6725 - val_loss: 0.5983 - val_acc: 0.6930
Epoch 12/200
3087/3087 [==============================] - 1s 454us/step - loss: 0.6186 - acc: 0.6709 - val_loss: 0.5902 - val_acc: 0.6891
Epoch 13/200
3087/3087 [==============================] - 1s 474us/step - loss: 0.6159 - acc: 0.6715 - val_loss: 0.5890 - val_acc: 0.6865
Epoch 14/200
3087/3087 [=============

Epoch 65/200
3087/3087 [==============================] - 1s 482us/step - loss: 0.6107 - acc: 0.6731 - val_loss: 0.5836 - val_acc: 0.6917
Epoch 66/200
3087/3087 [==============================] - 1s 465us/step - loss: 0.6125 - acc: 0.6715 - val_loss: 0.5835 - val_acc: 0.6917
Epoch 67/200
3087/3087 [==============================] - 2s 490us/step - loss: 0.6116 - acc: 0.6731 - val_loss: 0.5835 - val_acc: 0.6917
Epoch 68/200
3087/3087 [==============================] - 1s 472us/step - loss: 0.6136 - acc: 0.6725 - val_loss: 0.5835 - val_acc: 0.6917
Epoch 69/200
3087/3087 [==============================] - 1s 452us/step - loss: 0.6142 - acc: 0.6728 - val_loss: 0.5835 - val_acc: 0.6917
Epoch 70/200
3087/3087 [==============================] - 1s 472us/step - loss: 0.6104 - acc: 0.6722 - val_loss: 0.5835 - val_acc: 0.6917
Epoch 71/200
3087/3087 [==============================] - 1s 474us/step - loss: 0.6153 - acc: 0.6712 - val_loss: 0.5835 - val_acc: 0.6917
Epoch 72/200
3087/3087 [==========

3087/3087 [==============================] - 1s 407us/step - loss: 0.6125 - acc: 0.6722 - val_loss: 0.5830 - val_acc: 0.6930
Epoch 125/200
3087/3087 [==============================] - 1s 472us/step - loss: 0.6124 - acc: 0.6728 - val_loss: 0.5830 - val_acc: 0.6930
Epoch 126/200
3087/3087 [==============================] - 2s 502us/step - loss: 0.6139 - acc: 0.6706 - val_loss: 0.5830 - val_acc: 0.6930
Epoch 127/200
3087/3087 [==============================] - 1s 484us/step - loss: 0.6135 - acc: 0.6718 - val_loss: 0.5830 - val_acc: 0.6930
Epoch 128/200
3087/3087 [==============================] - 1s 486us/step - loss: 0.6134 - acc: 0.6722 - val_loss: 0.5829 - val_acc: 0.6930
Epoch 129/200
3087/3087 [==============================] - 2s 497us/step - loss: 0.6132 - acc: 0.6725 - val_loss: 0.5830 - val_acc: 0.6930
Epoch 130/200
3087/3087 [==============================] - 1s 468us/step - loss: 0.6137 - acc: 0.6728 - val_loss: 0.5829 - val_acc: 0.6930
Epoch 131/200
3087/3087 [================

3087/3087 [==============================] - 1s 460us/step - loss: 0.6128 - acc: 0.6718 - val_loss: 0.5826 - val_acc: 0.6930
Epoch 184/200
3087/3087 [==============================] - 1s 485us/step - loss: 0.6126 - acc: 0.6715 - val_loss: 0.5826 - val_acc: 0.6930
Epoch 185/200
3087/3087 [==============================] - 2s 490us/step - loss: 0.6121 - acc: 0.6718 - val_loss: 0.5826 - val_acc: 0.6930
Epoch 186/200
3087/3087 [==============================] - 1s 479us/step - loss: 0.6135 - acc: 0.6725 - val_loss: 0.5826 - val_acc: 0.6930
Epoch 187/200
3087/3087 [==============================] - 2s 490us/step - loss: 0.6147 - acc: 0.6718 - val_loss: 0.5826 - val_acc: 0.6930
Epoch 188/200
3087/3087 [==============================] - 2s 491us/step - loss: 0.6121 - acc: 0.6718 - val_loss: 0.5826 - val_acc: 0.6930
Epoch 189/200
3087/3087 [==============================] - 1s 459us/step - loss: 0.6141 - acc: 0.6715 - val_loss: 0.5826 - val_acc: 0.6930
Epoch 190/200
3087/3087 [================

3087/3087 [==============================] - 1s 465us/step - loss: 0.6171 - acc: 0.6693 - val_loss: 0.5914 - val_acc: 0.6891
Epoch 42/50
3087/3087 [==============================] - 1s 460us/step - loss: 0.6164 - acc: 0.6709 - val_loss: 0.5915 - val_acc: 0.6891
Epoch 43/50
3087/3087 [==============================] - 1s 478us/step - loss: 0.6143 - acc: 0.6709 - val_loss: 0.5914 - val_acc: 0.6891
Epoch 44/50
3087/3087 [==============================] - 1s 467us/step - loss: 0.6169 - acc: 0.6722 - val_loss: 0.5913 - val_acc: 0.6891
Epoch 45/50
3087/3087 [==============================] - 1s 478us/step - loss: 0.6166 - acc: 0.6709 - val_loss: 0.5913 - val_acc: 0.6891
Epoch 46/50
3087/3087 [==============================] - 1s 465us/step - loss: 0.6202 - acc: 0.6712 - val_loss: 0.5914 - val_acc: 0.6891
Epoch 47/50
3087/3087 [==============================] - 1s 470us/step - loss: 0.6182 - acc: 0.6706 - val_loss: 0.5914 - val_acc: 0.6891
Epoch 48/50
3087/3087 [==============================

In [3]:
print('Confusion Matrix for LSTM predictions:')
print(confusion_matrix(y_pred=y_classes, y_true=y_test.astype(int)))
print('Classification Report for LSTM predictions:')
print('-------------------------------------------')
print(classification_report(y_pred=y_classes, y_true=y_test.astype(int)))
print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test.astype(int))))

Confusion Matrix for LSTM predictions:
[[   1  368]
 [   1 1151]]
Classification Report for LSTM predictions:
-------------------------------------------
              precision    recall  f1-score   support

           0       0.50      0.00      0.01       369
           1       0.76      1.00      0.86      1152

   micro avg       0.76      0.76      0.76      1521
   macro avg       0.63      0.50      0.43      1521
weighted avg       0.70      0.76      0.65      1521

Accuracy score: 0.757396449704142


In [9]:
y_test.astype(int)

array([1, 0, 1, ..., 0, 1, 1])

In [11]:
pd.read_pickle("BBB_Test_Targets_1D.pkl").astype(int)

array([1, 0, 1, ..., 0, 1, 1])